In [1]:
import tensorflow
import cv2
from keras.models import load_model
import time
import random 

In [2]:
model = load_model('YOUR_MODEL.h5')

2022-03-15 00:36:59.914867: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import numpy as np

In [4]:
your_options = ['nothing', 'paper', 'rock', 'scissors']
def player_choice_prediction(prediction):  
                if prediction[0][0] > 0.5:
                    player_choice = your_options[0]
                elif prediction[0][1] > 0.5:
                    player_choice = your_options[1]
                elif prediction[0][2] > 0.5:
                    player_choice = your_options[2]
                else:
                    player_choice = your_options[3]
                return player_choice

def compare_ans(player_choice, player_score, comp_score):
                computer_choice = random.choice(['rock', 'paper', 'scissors', 'nothing'])
                if player_choice == 'paper' :
                    if computer_choice == 'paper' : 
                          message_1 = 'It is a draw'
                          comp_score = player_score = False
                    elif computer_choice == 'scissors':
                          message_1 = 'sorry you lost'
                          comp_score = True
                    elif computer_choice == 'rock':
                          message_1 = 'you won mate!'
                          player_score = True
                    else:
                          message_1 = 'please try again'
                          comp_score = player_score = False      
                elif player_choice == 'rock':
                    if computer_choice == 'scissors':
                          message_1 = 'you won mate!!'
                          player_score = True
                    elif computer_choice == 'rock':
                          message_1 = 'it is a draw'
                          comp_score = player_score = False
                    elif computer_choice == 'paper':
                          message_1 = 'sorry you lost'
                          comp_score = True
                    else:
                          message_1 = 'please try again'
                          comp_score = player_score = False   
                elif player_choice == 'scissors':
                    if computer_choice == 'paper':
                          message_1 = 'you won mate!'
                          player_score = True
                    elif computer_choice == 'rock':
                          message_1 = 'sorry you lost'
                          comp_score = True
                    elif computer_choice == 'scissors':
                          message_1 = 'it is a draw'
                          comp_score = player_score = False
                    else:
                          message_1 = 'please try again'
                          comp_score = player_score = False
                elif player_choice == 'nothing':
                    if computer_choice == 'rock' or 'paper' or 'scissors' or 'nothing':
                        message_1 = 'please try again'
                        comp_score = player_score = False
                return computer_choice, message_1, player_score, comp_score

cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
started = False
countdown = False
counter = 0
elapsed = 0
flag = True
comp_score_no = 0
player_score_no = 0

message = ''
while True:
    ret, frame= cap.read()
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    cv2.putText(frame,message, (30,30), cv2.FONT_HERSHEY_DUPLEX, 0.75, (0, 0, 255), 1)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image
    cv2.imshow('frame', frame)
    if not started:
        message = 'Press k to start'
    if cv2.waitKey(20) == ord('k'):
        if  not started:
            counter = time.time()
            started = True
            countdown = True
    if started:
        elapsed = 4 - (time.time() - counter)
        if not countdown:

                
            if int(elapsed) <= -6:
                    message = 'press o to play the next round'
            
            if cv2.waitKey(20) == ord('o'):
                    started = False 
                    elapsed = 0 
                    flag = True  
                    countdown = False   
        
        elif not flag:
            countdown = False
            comp_score = False
            player_score = False
            prediction = model.predict(data)
            player_choice = prediction.argmax
            player_func = player_choice_prediction(prediction)
            comp_func = compare_ans(player_func, player_score, comp_score) 
            if comp_func[2] == True:
                player_score_no += 1
                message = f'the computer showed {comp_func[0]}, and you showed {player_func}, so {comp_func[1]} And score = {player_score_no}.'
                if player_score_no == 3:
                    message = 'Marvellous you won 3 rounds!! press q to close the game or carry on.'
            elif comp_func[3] == True:
                comp_score_no += 1   
                message = f'the computer showed {comp_func[0]}, and you showed {player_func}, so {comp_func[1]} and computer score = {comp_score_no}.'
                if comp_score_no == 3:
                    message = 'The computer won 3 rounds :(, press q to close the game.'
            elif comp_func[2] == comp_func[3] == False:
                message = f'the computer showed {comp_func[0]}, and you showed {player_func}, so {comp_func[1]}.'
        if countdown:
            message = f'show your hand in {int(elapsed)} seconds'
            
            if int(elapsed) <= 0:
                flag = False

        

    if cv2.waitKey(1) & 0xFF == ord('q'):
       break

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()


2022-03-15 00:37:12.170060: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
